In [ ]:
#This file calulates the Kendall's Tau value based on the average score for each ligand
#Input is the csv files from the GLIDE(the ones Jessie formatted) this includes XP, XP_core, Holo, Holo_core, and SP_core
#GLIDE SP is not calculated in this file becuase that was the regular docking.
#The Kendall's Taus for average of GLIDE SP is found in "AvgScoreGlide"
#Data is in ~/Formatted_Data/LogFiles/GLIDE/
#outputs a file with the Kendall's Tau values of average rankings for GLIDE

In [ ]:
import numpy as np
import scipy.stats as stats

In [ ]:
GLIDE_types = ['holo', 'holo_core', 'SP', 'SP_core', 'XP', 'XP_core']
clustering = ['TICA', 'PCA','GROMOS', 'TICA_CBA', 'PCA_CBA','GROMOS_CBA']

In [ ]:
KT_file = open("/home/dhkumar/Formatted_Data/LogFiles/GLIDE/Kendalls_Tau_avg.txt","w+")
answer = open("/net/jam-amaro-shared/bccgc4/Answers.csv","r")
ranswer = answer.readlines()
answer.close()
for typ in GLIDE_types:
    for clust in clustering:
        file_name = typ + '_' + clust + '_scores.csv'
        path = '/home/dhkumar/Formatted_Data/LogFiles/GLIDE/' + typ + '/' + file_name
        with open(path, 'r') as input_file:
            content = input_file.readlines()
        content = content[11:]
        
        avg_score = []
        
        for lig in range(1,460):
            temp = [0,0,0,0,0,0,0,0,0,0]
            for line in content:
                lig_num = int(line.split(',')[0].split('_')[1])
                lig_score = float(line.split(',')[1])
                lig_clustnum = int(line.split(',')[2])
                if lig_num == lig:
                    temp[lig_clustnum - 1] = lig_score
            avg_score_temp = (temp[0] + temp[1] + temp[2] + temp[3] + temp[4] + temp[5] + temp[6] + temp[7] + temp[8] + temp[9])/ 10.0
            avg_score.append(avg_score_temp)
                    
        rankings = np.arange(1,460)
        rankings_scores = avg_score
        for index in range(1,459):
            value = rankings_scores[index] 
            r1 = rankings[index]
            i = index - 1
            while i>=0:
                r2 = rankings[i]
                if value < rankings_scores[i]: 
                    rankings_scores[i+1] = rankings_scores[i]   
                    rankings_scores[i] = value
                    rankings[i+1] = r2
                    rankings[i] = r1
                    i=i-1
                else:
                    break   
        
        rguess = []
        for i in range(459):
            rguess.append(('CatS_'+str(rankings[i])+','+ str(i+1) + ',' + str(rankings_scores[i])))        

        guesslist = []
        for i in rguess:
            entry = i.split(" ")
            guesslist.append(entry)
        answerlist=[]
        for i in ranswer:
            entry = i.split(" ")
            answerlist.append(entry)

        ans = []
        for k in answerlist:
            for h in k:
                h = h.split(",")
                num = h[1]
                ans.append(num)

        gues = []
        for y in answerlist:
            for u in y:
                u = u.split(',')
                for p in guesslist:
                    for v in p:
                        v = v.split(',')
                        if u[0] == v[0]:
                            gues.append(v[1])

        gues = [int(i) for i in gues]
        ans = [int(i) for i in ans]

        tau,p_value = stats.kendalltau(ans,gues)

        print(typ + '----' + clust + ': ' + str(tau))

        KT_file.write(typ + ' ' + clust + ": Kendall's Tau = " + str(tau) + "  p-value: " + str(p_value) + '\n')
KT_file.close()
